# Exercise 2: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. Submission includes this notebook only with the exercise number and your ID as the filename. For example: `hw2_123456789_987654321.ipynb` if you submitted in pairs and `hw2_123456789.ipynb` if you submitted the exercise alone.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
1. Write your functions in this notebook only. **Do not create Python modules and import them**.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. **Do not import anything else.**
1. Your code must run without errors. Make sure your `numpy` version is at least 1.15.4 and that you are using at least python 3.6. Changes of the configuration we provided are at your own risk. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Answers to qualitative questions should be written in **markdown** cells (with $\LaTeX$ support). Answers that will be written in commented code blocks will not be checked.

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Construct a decision tree algorithm.
4. Prune the tree to achieve better results.
5. Visualize your results.

# I have read and understood the instructions:  

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Warmup - OOP in python

Our desicion tree will be implemented using a dedicated python class. Python classes are very similar to classes in Java.


You can use the following [site](https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/) to learn about classes in python.

In [2]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)
    
    def __str__(self):
        # what a user wants to see
        return "({!r}, {!r})".format(self.data, self.children)

    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({!r}, {!r})".format(type(self).__name__, self.data, self.children)
        # !r calls repr() on the argument

In [3]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
print(n.children)

[Node(6, []), Node(7, [])]


## Data preprocessing

For the following exercise, we will use a dataset containing mushroom data `agaricus-lepiota.csv`. 

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous
one (=there are only two classes **edible** and **poisonous**). 
    
The dataset contains 8124 observations with 22 features:
1. cap-shape: bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
4. bruises: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g,green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

First, we will read and explore the data using pandas and the `.read_csv` method. Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [79]:
# load dataset
data = pd.read_csv('agaricus-lepiota.csv')

One of the advantages of the Decision Tree algorithm is that almost no preprocessing is required. However, finding missing values is always required.

In [81]:
#############################################################################
# TODO: Find the columns with missing values and remove them from the data.#
#############################################################################
data = data.dropna(axis=1)
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################

We will split the dataset to `Training` and `Testing` datasets.

In [60]:
from sklearn.model_selection import train_test_split
# Making sure the last column will hold the labels
X, y = data.drop('class', axis=1), data['class']
X = np.column_stack([X,y])
# split dataset using random_state to get the same split each time
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)

Training dataset shape:  (6093, 22)
Testing dataset shape:  (2031, 22)


In [6]:
y.shape

(8124,)

## Impurity Measures

Impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Implement the functions `calc_gini` and `calc_entropy`. You are encouraged to test your implementation (10 points).

In [61]:
def calc_gini(data):
    """
    Calculate gini impurity measure of a dataset.
 
    Input:
    - data: any dataset where the last column holds the labels.
 
    Returns the gini impurity.    
    """
    gini = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    data_labels = pd.Series(data[:, -1]).value_counts().values
    data_labels = (data_labels ** 2) / (data.shape[0] ** 2)
    gini = 1 - sum(data_labels)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return gini

In [62]:
def calc_entropy(data):
    """
    Calculate the entropy of a dataset.

    Input:
    - data: any dataset where the last column holds the labels.

    Returns the entropy of the dataset.    
    """
    entropy = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    data_labels = pd.Series(data[:, -1]).value_counts().values
    data_labels = data_labels / data.shape[0]
    data_labels *= np.log2(data_labels)
    entropy = (-1) * sum(data_labels)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return entropy

In [63]:
##### Your Tests Here #####
calc_gini(X), calc_entropy(X)

(0.4995636322379775, 0.9993703627906085)

## Goodness of Split

Given a feature the Goodnees of Split measures the reduction in the impurity if we split the data according to the feature.
$$
\Delta\varphi(S, A) = \varphi(S) - \sum_{v\in Values(A)} \frac{|S_v|}{|S|}\varphi(S_v)
$$

In our implementation the goodness_of_split function will return either the Goodness of Split or the Gain Ratio as learned in class. You'll control the return value with the `gain_ratio` parameter. If this parameter will set to False (the default value) it will return the regular Goodness of Split. If it will set to True it will return the Gain Ratio.
$$
GainRatio(S,A)=\frac{InformationGain(S,A)}{SplitInformation(S,A)}
$$
Where:
$$
InformationGain(S,A)=Goodness\ of\ Split\ calculated\ with\ Entropy\ as\ the\ Impurity\ function \\
SplitInformation(S,A)=- \sum_{a\in A} \frac{|S_a|}{|S|}\log\frac{|S_a|}{|S|}
$$
NOTE: you can add more parameters to the function and you can also add more returning variables (The given parameters and the given returning variable should not be touch). (10 Points)

In [64]:
def information_gain(data, feature, impurity_func):
    """
    Calculate the information gain of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.
    - impurity func: a function that calculates the impurity.

    Returns the information gain.
    """
    value_of_att_vec = np.array([])
    data_df = pd.DataFrame(data)
    feature_values = pd.Series(data[:, feature]).unique()

    for feature_val in feature_values:
        sub_np_mat = data_df.loc[
            data_df[feature] == feature_val].values  # sub-DataFrame where feature == i'th feature_value
        value_of_att_vec = np.append(value_of_att_vec,
                                     impurity_func(sub_np_mat) * (sub_np_mat.shape[0] / data.shape[0]))

    return impurity_func(data) - sum(value_of_att_vec)

In [65]:
def split_in_information(data, feature):
    """
    Calculate the information gain of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.

    Returns the split in information.
    """
    value_of_att_vec = np.array([])
    data_df = pd.DataFrame(data)
    feature_values = pd.Series(data[:, feature]).unique()

    for feature_val in feature_values:
        sub_np_mat = data_df.loc[
            data_df[feature] == feature_val].values  # sub-DataFrame where feature == i'th feature_value
        value_of_att_vec = np.append(value_of_att_vec,
                                     np.log2((sub_np_mat.shape[0] / data.shape[0])) * (
                                                 sub_np_mat.shape[0] / data.shape[0]))
    return (-1) * sum(value_of_att_vec)

In [66]:
def goodness_of_split(data, feature, impurity_func, gain_ratio=False):
    """
    Calculate the goodness of split of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.
    - impurity func: a function that calculates the impurity.
    - gain_ratio: goodness of split or gain ratio flag.

    Returns the goodness of split (or the Gain Ration).  
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    
    if gain_ratio:
        return information_gain(data, feature, calc_entropy) / split_in_information(data, feature)
    else:
        return information_gain(data, feature, impurity_func)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

In [67]:
# my test
print(goodness_of_split(X, 6, calc_gini, True))

0.09175333601458768


##### Building a Decision Tree

Use a Python class to construct the decision tree. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the feature and value used for the split and its children.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes categorical data. In this exercise, when splitting a node create the number of children needed according to the attribute unique values.

Complete the class `DecisionNode`. The structure of this class is entirely up to you. 

Complete the function `build_tree`. This function should get the training dataset and the impurity as inputs, initiate a root for the decision tree and construct the tree according to the procedure you learned in class. (30 points)

In [68]:
class DecisionNode:
    """
    This class will hold everything you require to construct a decision tree.
    The structure of this class is up to you. However, you need to support basic
    functionality as described above. It is highly recommended that you
    first read and understand the entire exercise before diving into this class.
    """

    def __init__(self, feature, value, data, used_features=[]):
        self.feature = feature
        self.value = value
        self.children = []
        self.data = data
        self.used_features = used_features

    def add_child(self, node):
        self.children.append(node)

    def is_pure(self):
        if len(np.unique(self.data[:, (self.data.shape[1] - 1)])) == 1:
            return True
        return False

    def is_leaf(self):
        if len(self.children) == 0:
            return True
        return False

    def __str__(self):
        # what a user wants to see
        return "({!r}, {!r}, {!r}, {!r})".format(self.feature, self.value, self.data.shape, self.children)

    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({!r}, {!r}, {!r}, {!r})".format(type(self).__name__, self.feature, self.value, self.data.shape,
                                                   self.children)
        # !r calls repr() on the argument

In [69]:
def best_split_attribute_index(data, node, impurity_func, gain_ratio=False):
    """
        this function iterates over the node's data features and returns the feature index
        for which the goodness_of_split value was the highest.

        Input:
        - data: the node's data   LAST COL IS TARGET !!
        - impurity: the chosen impurity measure
        - gain_ratio: goodness of split or gain ratio flag

        Output: index of best split attribute.
        """
    best_goodness_value = -1
    for feature_index in range(data.shape[1] - 1):
        if feature_index not in node.used_features:
            temp_goodness_value = goodness_of_split(data, feature_index, impurity_func, gain_ratio)
            if temp_goodness_value > best_goodness_value:
                best_goodness_value = temp_goodness_value
                best_goodness_index = feature_index
    return best_goodness_index

In [70]:
def split_data_according_to_feature_values(data_to_split, feature_index):
    """
    this function splits the data according to a specific feature, unique values

    Input:
    - data_to_split: the dataset to split
    - feature_index:  the feature index to split on

    Output: python dictionary where keys are the attributes values and the values are the splitted data according to them
    """
    data_to_split_df = pd.DataFrame(data_to_split)
    best_feature_values = data_to_split[:, feature_index]
    splitted_data_dict = {}
    for val in (np.unique(best_feature_values)):
        splitted_data = data_to_split_df[data_to_split_df[feature_index] == val]
        splitted_data_dict[val] = splitted_data.values
    return splitted_data_dict

In [71]:
def build_tree(data, impurity, used_features=[], gain_ratio=False, chi=1, max_depth=1000, feature_value=None):
    """
    Build a tree using the given impurity measure and training dataset.
    You are required to fully grow the tree until all leaves are pure.

    Input:
    - data: the training dataset.
    - impurity: the chosen impurity measure. Notice that you can send a function
                as an argument in python.
    - gain_ratio: goodness of split or gain ratio flag
    - chi: chi square p-value cut off (1 means no pruning)
    - max_depth: the allowable depth of the tree

    Output: the root node of the tree.
    """
    root = DecisionNode(feature=None, value=feature_value, data=data, used_features=used_features.copy())
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if not root.is_pure() and (max_depth > 1):
        root.feature = best_split_attribute_index(data=data, node=root, impurity_func=impurity, gain_ratio=gain_ratio)
        root.used_features.append(root.feature)
        attribute_split_values_dict = split_data_according_to_feature_values(data_to_split=data, feature_index=root.feature)
        for val in attribute_split_values_dict.keys():
            root.children.append(build_tree(data=attribute_split_values_dict[val], used_features=root.used_features.copy(), impurity=impurity, gain_ratio=gain_ratio, chi=chi, max_depth=(max_depth - 1), feature_value=val))
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return root

In [72]:
# python supports passing a function as an argument to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini) # gini and goodness of split
tree_entropy = build_tree(data=X_train, impurity=calc_entropy) # entropy and goodness of split
tree_entropy_gain_ratio = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True) # entropy and gain ratio

## Tree evaluation

Complete the functions `predict` and `calc_accuracy`. (10 points)

In [44]:
def predict(node, instance):
    """
    Predict a given instance using the decision tree

    Input:
    - root: the root of the decision tree.
    - instance: an row vector from the dataset. Note that the last element
                of this vector is the label of the instance.

    Output: the prediction of the instance.
    """
    pred = None
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    while not node.is_leaf():
        for child in node.children:
            if instance[node.feature] == child.value:
                node = child
                break
    results = split_data_according_to_feature_values(data_to_split=node.data, feature_index=(node.data.shape[1] - 1))
    if len(list(results.keys())) == 1:
        return list(results.keys())[0]
    else:
        p_num = results['p'].shape[0]
        e_num = results['e'].shape[0]
        if e_num > p_num:
            return 'e'
        else:
            return 'p'

In [41]:
def calc_accuracy(node, dataset):
    """
    Predict a given dataset using the decision tree

    Input:
    - node: a node in the decision tree.
    - dataset: the dataset on which the accuracy is evaluated

    Output: the accuracy of the decision tree on the given dataset (%).
    """
    accuracy = 0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    true_class_count = 0
    for inst in dataset:
        if predict(node=node, instance=inst) == inst[(len(inst) - 1)]:
            true_class_count = true_class_count + 1
    accuracy = (true_class_count / dataset.shape[0]) * 100
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return accuracy

After building the three trees using the training set, you should calculate the accuracy on the test set. For each tree print the training and test accuracy. Select the tree that gave you the best test accuracy. For the rest of the exercise, use that tree (when you asked to build another tree use the same impurity function and same gain_ratio flag). 

In [ ]:
#### Your code here ####

## Post pruning

Iterate over all nodes in the tree that have at least a single child which is a leaf. For each such node, replace it with its most popular class. Calculate the accuracy on the testing dataset, pick the node that results in the highest testing accuracy and permanently change it in the tree. Repeat this process until you are left with a single node in the tree (the root). Finally, create a plot of the training and testing accuracies as a function of the number of nodes in the tree. (15 points)

In [ ]:
#### Your code here ####

## Chi square pre-pruning

Consider the following p-value cut-off values: [1 (no pruning), 0.5, 0.25, 0.1, 0.05, 0.0001 (max pruning)]. For each value, construct a tree and prune it according to the cut-off value. Next, calculate the training and testing accuracy. On a single plot, draw the training and testing accuracy as a function of the tuple (p-value, tree depth). Mark the best result on the graph with red circle. (15 points)

In [ ]:
### Chi square table values ###
# The first key is the degree of freedom 
# The second key is the p-value cut-off
# The values are the chi-statistic that you need to use in the pruning
chi_table = {1: {0.5 : 0.45,
                 0.25 : 1.32,
                 0.1 : 2.71,
                 0.05 : 3.84,
                 0.0001 : 100000},
             2: {0.5 : 1.39,
                 0.25 : 2.77,
                 0.1 : 4.60,
                 0.05 : 5.99,
                 0.0001 : 100000},
             3: {0.5 : 2.37,
                 0.25 : 4.11,
                 0.1 : 6.25,
                 0.05 : 7.82,
                 0.0001 : 100000},
             4: {0.5 : 3.36,
                 0.25 : 5.38,
                 0.1 : 7.78,
                 0.05 : 9.49,
                 0.0001 : 100000},
             5: {0.5 : 4.35,
                 0.25 : 6.63,
                 0.1 : 9.24,
                 0.05 : 11.07,
                 0.0001 : 100000},
             6: {0.5 : 5.35,
                 0.25 : 7.84,
                 0.1 : 10.64,
                 0.05 : 12.59,
                 0.0001 : 100000},
             7: {0.5 : 6.35,
                 0.25 : 9.04,
                 0.1 : 12.01,
                 0.05 : 14.07,
                 0.0001 : 100000},
             8: {0.5 : 7.34,
                 0.25 : 10.22,
                 0.1 : 13.36,
                 0.05 : 15.51,
                 0.0001 : 100000},
             9: {0.5 : 8.34,
                 0.25 : 11.39,
                 0.1 : 14.68,
                 0.05 : 16.92,
                 0.0001 : 100000},
             10: {0.5 : 9.34,
                  0.25 : 12.55,
                  0.1 : 15.99,
                  0.05 : 18.31,
                  0.0001 : 100000},
             11: {0.5 : 10.34,
                  0.25 : 13.7,
                  0.1 : 17.27,
                  0.05 : 19.68,
                  0.0001 : 100000}}

In [ ]:
#### Your code here ####

Build the best 2 trees:
1. tree_max_depth - the best tree according to max_depth pruning
1. tree_chi - the best tree according to chi square pruning

In [ ]:
#### Your code here ####

## Number of Nodes

Of the two trees above we will choose the one with fewer nodes. Complete the function counts_nodes and print the number of nodes in each tree. (5 points) 

In [ ]:
def count_nodes(node):
    """
    Count the number of node in a given tree
 
    Input:
    - node: a node in the decision tree.
 
    Output: the number of node in the tree.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    pass
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    

## Print the tree

Complete the function `print_tree` and execute it on your chosen tree. Your tree should be visualized clearly. You can use the following example as a reference:
```
[ROOT, feature=X0],
  [X0=a, feature=X2]
    [X2=c, leaf]: [{1.0: 10}]
    [X2=d, leaf]: [{0.0: 10}]
  [X0=y, feature=X5], 
    [X5=a, leaf]: [{1.0: 5}]
    [X5=s, leaf]: [{0.0: 10}]
  [X0=e, leaf]: [{0.0: 25, 1.0: 50}]
```
In each brackets:
* The first argument is the parent feature with the value that led to current node
* The second argument is the selected feature of the current node
* If the current node is a leaf, you need to print also the labels and their counts

(5 points)

In [73]:
def print_tabs(depth):
    for i in range(0, depth):
        print("  ", end="")

In [83]:
def print_tree(node, depth=0, parent_feature='ROOT', max_depth=5):
    '''
    prints the tree according to the example above

    Input:
    - node: a node in the decision tree

    This function has no return value
    '''
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if node == None:
        return
    poisonous = 0
    healthy = 0
    for row in node.data:
        if row[21] == 'p':
            poisonous = poisonous + 1
        else:
            healthy = healthy + 1
    if depth == 0:
        print_tabs(depth)
        print("[{}, feature=X{}]".format(parent_feature, node.feature), end="")
        print("")
    elif depth == max_depth or node.is_pure(): 
        if poisonous == 0:
            brackets = "[{{0.0: {}}}]".format(healthy)
        elif healthy == 0:
            brackets = "[{{0.0: {}}}]".format(poisonous)
        else:
            brackets = "[{{1.0: {}, 0.0: {}}}"
        print_tabs(depth)
        print("[{}={}, leaf]: {}]".format(parent_feature, node.value, brackets, poisonous, healthy), end="")
        print("")
    elif node.feature is not None:
        print_tabs(depth)
        print("[{}={}, feature=X{}]".format(parent_feature, node.value, node.feature), end="")
        print("")
    for child in node.children:
        print_tree(child, depth + 1, "X" + str(node.feature))
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

In [84]:
print_tree(tree_entropy)

[ROOT, feature=X4]
  [X4=a, feature=X2]
    [X2=n, feature=X8]
      [X8=n, feature=X19]
        [X19=s, feature=X0]
          [X0=f, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X18=k, leaf]: [{0.0: 1}]]
            [X18=n, feature=X20]
              [X20=g, leaf]: [{0.0: 1}]]
              [X20=p, leaf]: [{0.0: 1}]]
          [X0=x, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X1=y, feature=X3]
              [X3=t, feature=X5]
                [X5=f, feature=X6]
                  [X6=c, feature=X7]
                    [X7=b, feature=X9]
                      [X9=e, feature=X10]
                        [X10=s, feature=X11]
                          [X11=y, feature=X12]
                            [X12=w, feature=X13]
                              [X13=w, feature=X14]
                                [X14=p, feature=X15]
                                  [X15=w, feature=X16]
                                    [X16=o, feature=X17]
                                      [X17=p, feature=X18]
  

              [X20=d, feature=X13]
                [X13=b, leaf]: [{0.0: 2}]]
                [X13=n, feature=X12]
                  [X12=b, leaf]: [{0.0: 1}]]
                  [X12=n, leaf]: [{0.0: 1}]]
                  [X12=p, leaf]: [{0.0: 1}]]
                [X13=p, leaf]: [{0.0: 1}]]
              [X20=g, feature=X12]
                [X12=b, feature=X13]
                  [X13=b, leaf]: [{0.0: 1}]]
                  [X13=n, leaf]: [{0.0: 1}]]
                  [X13=p, leaf]: [{0.0: 1}]]
                [X12=n, leaf]: [{0.0: 3}]]
                [X12=p, leaf]: [{0.0: 3}]]
              [X20=p, feature=X13]
                [X13=b, feature=X12]
                  [X12=b, leaf]: [{0.0: 1}]]
                  [X12=n, leaf]: [{0.0: 1}]]
                  [X12=p, leaf]: [{0.0: 1}]]
                [X13=n, leaf]: [{0.0: 2}]]
                [X13=p, feature=X12]
                  [X12=b, leaf]: [{0.0: 1}]]
                  [X12=n, leaf]: [{0.0: 1}]]
                  [X12=p, leaf]: [{0.

            [X11=f, leaf]: [{0.0: 1}]]
            [X11=s, feature=X10]
              [X10=f, feature=X20]
                [X20=g, leaf]: [{0.0: 1}]]
                [X20=u, leaf]: [{0.0: 1}]]
              [X10=s, leaf]: [{0.0: 1}]]
          [X19=v, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X20=g, leaf]: [{0.0: 4}]]
            [X20=u, feature=X10]
              [X10=f, leaf]: [{0.0: 1}]]
              [X10=s, feature=X11]
                [X11=f, leaf]: [{0.0: 1}]]
                [X11=s, leaf]: [{0.0: 1}]]
        [X0=x, feature=X10]
          [X10=f, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X11=f, feature=X19]
              [X19=s, leaf]: [{0.0: 2}]]
              [X19=v, leaf]: [{0.0: 1}]]
            [X11=s, leaf]: [{0.0: 3}]]
          [X10=s, leaf]: [{0.0: 7}]]
    [X2=n, feature=X20]
      [X20=d, feature=X11]
        [X11=k, feature=X1]
          [X1=s, leaf]: [{0.0: 18}]]
          [X1=y, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X12=p, leaf]: [{0.0: 7}]]
            [X1

              [X13=b, feature=X0]
                [X0=f, feature=X1]
                  [X1=f, leaf]: [{0.0: 1}]]
                  [X1=y, leaf]: [{0.0: 1}]]
                [X0=x, leaf]: [{0.0: 2}]]
              [X13=n, leaf]: [{0.0: 2}]]
              [X13=p, leaf]: [{0.0: 3}]]
          [X20=g, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X8=g, leaf]: [{0.0: 10}]]
            [X8=h, feature=X0]
              [X0=f, leaf]: [{0.0: 5}]]
              [X0=x, feature=X1]
                [X1=f, leaf]: [{0.0: 1}]]
                [X1=y, feature=X13]
                  [X13=b, leaf]: [{0.0: 1}]]
                  [X13=p, leaf]: [{0.0: 1}]]
            [X8=p, leaf]: [{0.0: 7}]]
          [X20=p, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X8=g, leaf]: [{0.0: 6}]]
            [X8=h, feature=X13]
              [X13=b, feature=X0]
                [X0=f, leaf]: [{0.0: 1}]]
                [X0=x, feature=X1]
                  [X1=f, leaf]: [{0.0: 1}]]
                  [X1=y, leaf]: [{0.0: 1}]]
     

    [X0=k, feature=X2]
      [X2=c, leaf]: [{0.0: 4}]]
      [X2=e, feature=X5]
        [X5=a, feature=X8]
          [X8=w, leaf]: [{0.0: 1}]]
          [X8=y, leaf]: [{0.0: 1}]]
        [X5=f, leaf]: [{0.0: 2}]]
      [X2=n, leaf]: [{0.0: 3}]]
    [X0=x, leaf]: [{0.0: 7}]]
  [X4=n, feature=X18]
    [X18=b, feature=X0]
      [X0=b, leaf]: [{0.0: 10}]]
      [X0=f, leaf]: [{0.0: 8}]]
      [X0=k, feature=X8]
        [X8=n, leaf]: [{0.0: 3}]]
        [X8=o, leaf]: [{0.0: 1}]]
        [X8=y, leaf]: [{0.0: 3}]]
      [X0=x, feature=X8]
        [X8=n, feature=X19]
          [X19=c, leaf]: [{0.0: 1}]]
          [X19=v, leaf]: [{0.0: 2}]]
        [X8=o, leaf]: [{0.0: 4}]]
        [X8=y, leaf]: [{0.0: 4}]]
    [X18=h, feature=X2]
      [X2=r, feature=X8]
        [X8=h, feature=X0]
          [X0=f, leaf]: [{0.0: 2}]]
          [X0=x, leaf]: [{0.0: 1}]]
        [X8=p, feature=X0]
          [X0=f, leaf]: [{0.0: 2}]]
          [X0=x, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X19=v, leaf]: [{0.0: 

...

                [X0=x, leaf]: [{0.0: 1}]]
              [X2=n, feature=X0]
                [X0=f, feature=X1]
                  [X1=f, leaf]: [{0.0: 1}]]
                  [X1=y, leaf]: [{0.0: 1}]]
                [X0=x, leaf]: [{0.0: 2}]]
            [X19=y, leaf]: [{0.0: 7}]]
        [X12=w, feature=X0]
          [X0=f, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X2=e, feature=X19]
              [X19=v, feature=X13]
                [X13=g, leaf]: [{0.0: 1}]]
                [X13=p, leaf]: [{0.0: 2}]]
                [X13=w, feature=X1]
                  [X1=f, leaf]: [{0.0: 1}]]
                  [X1=y, leaf]: [{0.0: 1}]]
              [X19=y, leaf]: [{0.0: 5}]]
            [X2=g, leaf]: [{0.0: 6}]]
            [X2=n, feature=X13]
              [X13=g, leaf]: [{0.0: 4}]]
              [X13=p, feature=X1]
                [X1=f, leaf]: [{0.0: 1}]]
                [X1=y, leaf]: [{0.0: 1}]]
              [X13=w, leaf]: [{0.0: 2}]]
          [X0=x, leaf]: [{0.0: 25}]]
      [X8=w, feature=

                  [X0=x, feature=X3]
                    [X3=f, leaf]: [{0.0: 1}]]
                    [X3=t, leaf]: [{0.0: 1}]]
          [X1=s, leaf]: [{0.0: 6}]]
          [X1=y, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X12=g, feature=X2]
              [X2=e, feature=X0]
                [X0=f, leaf]: [{0.0: 3}]]
                [X0=x, feature=X13]
                  [X13=g, leaf]: [{0.0: 1}]]
                  [X13=p, leaf]: [{0.0: 1}]]
              [X2=g, leaf]: [{0.0: 4}]]
              [X2=n, leaf]: [{0.0: 4}]]
            [X12=p, leaf]: [{0.0: 13}]]
            [X12=w, leaf]: [{0.0: 12}]]
        [X19=y, feature=X2]
          [X2=e, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X12=g, leaf]: [{0.0: 8}]]
            [X12=p, feature=X13]
              [X13=g, leaf]: [{0.0: 1}]]
              [X13=p, feature=X0]
                [X0=f, feature=X1]
                  [X1=f, leaf]: [{0.0: 1}]]
                  [X1=y, leaf]: [{0.0: 1}]]
                [X0=x, leaf]: [{0.0: 2}]]
        

              [X2=e, leaf]: [{0.0: 4}]]
              [X2=g, feature=X19]
                [X19=v, leaf]: [{0.0: 2}]]
                [X19=y, leaf]: [{0.0: 1}]]
              [X2=n, feature=X19]
                [X19=v, leaf]: [{0.0: 1}]]
                [X19=y, leaf]: [{0.0: 2}]]
      [X8=w, feature=X12]
        [X12=g, feature=X0]
          [X0=f, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X13=g, feature=X2]
              [X2=e, feature=X1]
                [X1=f, leaf]: [{0.0: 2}]]
                [X1=y, feature=X19]
                  [X19=v, leaf]: [{0.0: 1}]]
                  [X19=y, leaf]: [{0.0: 1}]]
              [X2=g, leaf]: [{0.0: 2}]]
              [X2=n, feature=X1]
                [X1=f, feature=X19]
                  [X19=v, leaf]: [{0.0: 1}]]
                  [X19=y, leaf]: [{0.0: 1}]]
                [X1=y, leaf]: [{0.0: 2}]]
            [X13=p, feature=X2]
              [X2=e, feature=X19]
                [X19=v, leaf]: [{0.0: 1}]]
                [X19=y, leaf]: [{0.0

                    [X19=n, leaf]: [{0.0: 1}]]
                    [X19=s, leaf]: [{0.0: 1}]]
                  [X11=s, leaf]: [{0.0: 2}]]
            [X0=x, leaf]: [{0.0: 13}]]
          [X10=s, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X11=k, feature=X0]
              [X0=b, feature=X1]
                [X1=f, leaf]: [{0.0: 4}]]
                [X1=s, leaf]: [{0.0: 1}]]
              [X0=k, leaf]: [{0.0: 7}]]
              [X0=x, leaf]: [{0.0: 8}]]
            [X11=s, feature=X1]
              [X1=f, feature=X0]
                [X0=b, leaf]: [{0.0: 2}]]
                [X0=k, feature=X2]
                  [X2=g, leaf]: [{0.0: 2}]]
                  [X2=w, leaf]: [{0.0: 1}]]
                [X0=x, feature=X2]
                  [X2=g, feature=X19]
                    [X19=n, leaf]: [{0.0: 1}]]
                    [X19=s, leaf]: [{0.0: 1}]]
                  [X2=w, leaf]: [{0.0: 1}]]
              [X1=s, leaf]: [{0.0: 11}]]
      [X20=l, feature=X2]
        [X2=c, feature=X0]
          

          [X10=k, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X11=k, feature=X13]
              [X13=p, feature=X2]
                [X2=e, leaf]: [{0.0: 1}]]
                [X2=n, feature=X12]
                  [X12=p, leaf]: [{0.0: 1}]]
                  [X12=w, leaf]: [{0.0: 1}]]
              [X13=w, leaf]: [{0.0: 3}]]
            [X11=s, leaf]: [{0.0: 5}]]
          [X10=s, leaf]: [{0.0: 14}]]
        [X0=x, feature=X13]
          [X13=p, leaf]: [{0.0: 13}]]
          [X13=w, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X12=p, leaf]: [{0.0: 6}]]
            [X12=w, feature=X10]
              [X10=k, leaf]: [{0.0: 2}]]
              [X10=s, leaf]: [{0.0: 1}]]
      [X20=l, feature=X2]
        [X2=e, feature=X0]
          [X0=f, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X11=k, leaf]: [{0.0: 6}]]
            [X11=s, feature=X12]
              [X12=p, leaf]: [{0.0: 2}]]
              [X12=w, leaf]: [{0.0: 1}]]
          [X0=k, leaf]: [{{1.0: {}, 0.0: {}}}]
            [X11=k, feature=X1